In [1]:
#this is to import the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import queue
import heapq

In [2]:

#class for the environment of the n^2 -1 puzzle
#this is also going to have the function for the parity check
class puzzle : 
    def __init__(self):
        n = int(input("Enter the dimension of the matrix : "))
        self.dimension = n
        self.mat = np.zeros((self.dimension , self.dimension))
        self.x1 = -1
        self.y1 = -1
        self.x2 = -1
        self.y2 = -1
        self.goal = np.arange(1,n*n+1).reshape((n , n))
        self.goal[self.dimension - 1, self.dimension - 2] = self.dimension * self.dimension
    #this is to take the matrix as the input 
    #in order to take the blank we take * as the input
    def enter_matrix(self):
        first = False
        print("Enter the elements of the matrix : ")
        for i in range(self.dimension):
            temp  = input().strip().split()
            for j , x in zip(range(self.dimension) , temp):
                if (x == '*' and first == False):
                    self.mat[i , j] = self.dimension * self.dimension -1
                    self.x1 = i
                    self.y1 = j
                    first = True
                elif (x == '*' and first == True):
                    self.mat[i , j] = self.dimension * self.dimension
                    self.x2 = i
                    self.y2 = j
                else:
                    self.mat[i , j] = int(x)
    
    
    def duplicate(self , matrix):
            self.mat = np.copy(matrix)
            first = False
            for i in range(self.dimension):
                for j in range(self.dimension):
                    if (self.dimension * self.dimension - 1 == self.mat[i , j]):
                        self.x1 = i
                        self.y1 = j
                        first = True
                    elif (self.dimension * self.dimension == self.mat[i , j]):
                        self.x2 = i;
                        self.y2 = j;
    #this is the function to define the movements of the blank space
    #blank 0 is the self.dimension * self.dimension - 1
    def left (self , blank_number):
        if (blank_number == 0):
            if (self.y1 == 0):
                return False
            if (self.mat[self.x1 , self.y1 - 1] == self.dimension * self.dimension):
                return False
            else:
                self.mat[self.x1 , self.y1] = self.mat[self.x1, self.y1 - 1]
                self.mat[self.x1, self.y1 - 1] = self.dimension * self.dimension - 1
                self.y1 = self.y1 - 1
                return True
        elif (blank_number == 1 ):
            if (self.y2 == 0):
                return False
            if (self.mat[self.x2 , self.y2 - 1] == self.dimension * self.dimension - 1):
                return False
            else:
                self.mat[self.x2 , self.y2] = self.mat[self.x2, self.y2 - 1]
                self.mat[self.x2, self.y2 - 1] = self.dimension * self.dimension
                self.y2 = self.y2 - 1
                return True
            
        
    def right(self , blank_number):
        if (blank_number == 0):
            if (self.y1 == self.dimension - 1 ):
                return False
            if (self.mat[self.x1 , self.y1 + 1] == self.dimension * self.dimension):
                return False
            else:
                self.mat[self.x1 , self.y1] = self.mat[self.x1 , self.y1 + 1]
                self.mat[self.x1 , self.y1 + 1] = self.dimension * self.dimension - 1
                self.y1 = self.y1 + 1
                return True
        elif (blank_number == 1):
            if (self.y2 == self.dimension - 1 ):
                return False
            if (self.mat[self.x2 , self.y2 + 1] == self.dimension * self.dimension - 1):
                return False
            else:
                self.mat[self.x2 , self.y2] = self.mat[self.x2 , self.y2 + 1]
                self.mat[self.x2 , self.y2 + 1] = self.dimension * self.dimension
                self.y2 = self.y2 + 1
                return True
            
    def up(self , blank_number):
        if (blank_number == 0):
            if (self.x1 ==0 ):
                return False
            if (self.mat[self.x1 -1 , self.y1] == self.dimension * self.dimension):
                return False
            else:
                self.mat[self.x1 , self.y1] = self.mat[self.x1 - 1 , self.y1]
                self.mat[self.x1 - 1  , self.y1 ] = self.dimension * self.dimension - 1
                self.x1 = self.x1 - 1
                return True
        elif (blank_number == 1):
            if (self.x2 == 0):
                return False
            if (self.mat[self.x2 - 1 , self.y2] == self.dimension * self.dimension - 1):
                return False
            else:
                self.mat[self.x2 , self.y2] = self.mat[self.x2 - 1 , self.y2]
                self.mat[self.x2 - 1  , self.y2 ] = self.dimension * self.dimension
                self.x2 = self.x2 - 1
                return True
            
    def down(self , blank_number):
        if (blank_number == 0 ):
            
            if (self.x1 == self.dimension - 1):
                return False
            if (self.mat[self.x1 + 1 , self.y1] == self.dimension * self.dimension):
                #print ("hi2")
                return False
            else:
                self.mat[self.x1 , self.y1] = self.mat[self.x1 + 1 , self.y1]
                self.mat[self.x1 + 1, self.y1 ] = self.dimension * self.dimension - 1
                self.x1 = self.x1  + 1
                return True
        elif (blank_number == 1):
            
            if (self.x2 == self.dimension - 1):
                return False
            if (self.mat[self.x2 + 1 , self.y2] == self.dimension * self.dimension - 1):
                return False
            else:
                self.mat[self.x2 , self.y2] = self.mat[self.x2 + 1 , self.y2]
                self.mat[self.x2 + 1, self.y2 ] = self.dimension * self.dimension
                self.x2 = self.x2  + 1
                return True
            
    #now this will be for the heuristic
    #this is the manhattan heuristic that predicts how many moves more to go
    #how far the value of the tile is from the target
    def h_manhattan (self):
        h = 0
        first = False
        for i in range(self.dimension):
            for j in range(self.dimension):
                p1 = int(self.mat[i , j]) - 1
                x1 = p1 // self.dimension
                y1 = p1 % self.dimension
                h += abs (x1 - i) + abs (y1 - j)
                               
        return int(h)
    

In [9]:
#initialising the value and then taking the matrix as the input
#this is to take the matrix and then make it reach the goal state
#note having the value of 9 indicates the blank space
#8 represents the first blank space4
env = puzzle()
env.enter_matrix()
print("The matrix is : ")
print(env.mat)

Enter the dimension of the matrix : 4
Enter the elements of the matrix : 
1 2 3 4
5 6 7 8
9 10 12 11
13 14 * *
The matrix is : 
[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 12. 11.]
 [13. 14. 15. 16.]]


In [ ]:
#### ans = env.right(1)
print(env.mat ,ans)

In [19]:
print(env.dimension , env.y2)

3 2


In [10]:
ans = env.left(1)
print(env.mat ,ans)

[[1. 2. 3.]
 [4. 5. 9.]
 [6. 7. 8.]] False


In [22]:
#printing the manhattan heuristic for this case
env.h_manhattan()

5

In [43]:
#printing the posiotion of the blanks
print(env.x1 , env.y1)
print(env.x2 , env.y2)

2 1
2 2


In [10]:
#this is to print the goal state
print(env.goal)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [34]:
env.left(1)
print(env.x2 , env.y2)

2 2


In [10]:
#solving using the A* heuristic
def heuristics():
    current_string = np.array_str(env.mat)   #this is the matrix to convert the matrix in the string form to keep 
                                             #track of the explored states
    
    d = dict()                #this is to store the node
                              #first element is the matrix which is the parent of the string
                              #second is the action on the parent that leads to the current node
                              #third is the total cost to go till now
                              #fourth is the heuristic
                    
    q = []                    #this is the list and in some sense we are going to treat it as the priority queue
    heapq.heapify (q)
    
    explored = []         #this is to maintain the set of the explored states
                             #this is to initialised to empty
        
    frontier = []            #frontier is the list that is used to keep track of the elements that are there in the queue
    
    
    d[current_string] = ("-1" , "-1" , 0 , env.h_manhattan())         #this is to put the current string in the state and initialise all it parameters
    ans = False              #this is to see if the solution is found or not
        
    counter = 0              #to see how many steps have been taken to solve the problem
    counter_prior = 0
    heapq.heappush(q , (d[current_string][2] + d[current_string][3] , counter_prior ,np.copy(env.mat) ) )   #this is to put the value of the heuristic in the queue
    frontier.append (current_string)
    
    while len(q) > 0 and ans == False:
        
    
        #print(frontier)
        #this is to print the number of steps
        counter += 1             #to count the number of steps taken
        print("Number of steps : " , counter)
        
        
        #the element that is popped out from the heap have the fist element as the total estimated cost 
        #and the third element as the state from which it has beee taken
        temp = heapq.heappop (q)      #popping the element out of the queue
        
        
        
        
        matrix = temp[2]                          #taking the matrix out of it
        matrix_string = np.array_str (matrix)     #converting the matrix into the string
        frontier.remove(matrix_string)
        env.duplicate(matrix)                     #creating the duplicate of the environment
        #print (matrix)
    
        for_check = np.copy (env.mat)
        for_check[env.x1 , env.y1] = env.dimension * env.dimension
        if (np.array_equal(for_check , env.goal)):  #to check if this is the goal state if yes break out of the loop
            ans = True
            break

        explored.append (matrix_string)              #to say that this node is explored
        
        dist = d[matrix_string][2] + d[matrix_string][3]      #getting the actual value out of the d
        if (temp[0] > dist):                      #if the value in the queue is greater than the current distance then discarding it
            continue                              #note this is for the lazy detection
        
        #this is to carry out each action
        #this is to explore the current string
        #moving just the first blank
        if (env.left(0) == True):
            current_string = np.array_str (env.mat)
            if (current_string not in explored and current_string not in frontier):
                #this is to update the value of the dictionary
                d[current_string] = (matrix_string , "left b0" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
            elif (current_string in frontier and d[current_string][2] > d[matrix_string][2] + 1):
                d[current_string] = (matrix_string , "left b0" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
                
            env.right(0)
            
        if (env.left(1) == True):
            current_string = np.array_str (env.mat)
            if (current_string not in explored and current_string not in frontier):
                #this is to update the value of the dictionary
                d[current_string] = (matrix_string , "left b1" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
            elif (current_string in frontier and d[current_string][2] > d[matrix_string][2] + 1):
                d[current_string] = (matrix_string , "left b1" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
                
            env.right(1)
            
        if (env.right(0) == True):
            current_string = np.array_str (env.mat)
            if (current_string not in explored and current_string not in frontier):
                #this is to update the value of the dictionary
                d[current_string] = (matrix_string , "right b0" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
            elif (current_string in frontier and d[current_string][2] > d[matrix_string][2] + 1):
                d[current_string] = (matrix_string , "right b0" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
                
            env.left(0)
        
        if (env.right(1) == True):
            current_string = np.array_str (env.mat)
            if (current_string not in explored and current_string not in frontier):
                #this is to update the value of the dictionary
                d[current_string] = (matrix_string , "right b1" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
            elif (current_string in frontier and d[current_string][2] > d[matrix_string][2] + 1):
                d[current_string] = (matrix_string , "right b1" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
                
            env.left(1)
            
        if (env.up(0) == True):
            current_string = np.array_str (env.mat)
            if (current_string not in explored and current_string not in frontier):
                #this is to update the value of the dictionary
                d[current_string] = (matrix_string , "up b0" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
            elif (current_string in frontier and d[current_string][2] > d[matrix_string][2] + 1):
                d[current_string] = (matrix_string , "up b0" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
                
            env.down(0)
            
        if (env.up(1) == True):
            current_string = np.array_str (env.mat)
            if (current_string not in explored and current_string not in frontier):
                #this is to update the value of the dictionary
                d[current_string] = (matrix_string , "up b1" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
            elif (current_string in frontier and d[current_string][2] > d[matrix_string][2] + 1):
                d[current_string] = (matrix_string , "up b1" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
                
            env.down(1)
            
        if (env.down(0) == True):
            current_string = np.array_str (env.mat)
            if (current_string not in explored and current_string not in frontier):
                #this is to update the value of the dictionary
                d[current_string] = (matrix_string , "down b0" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
            elif (current_string in frontier and d[current_string][2] > d[matrix_string][2] + 1):
                d[current_string] = (matrix_string , "down b0" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
                
            env.up(0)
        
        if (env.down(1) == True):
            #print("hi")
            #print(env.mat)
            current_string = np.array_str (env.mat)
            if (current_string not in explored and current_string not in frontier):
                #this is to update the value of the dictionary
                d[current_string] = (matrix_string , "down b1" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
            elif (current_string in frontier and d[current_string][2] > d[matrix_string][2] + 1):
                d[current_string] = (matrix_string , "down b1" , d[matrix_string][2] + 1 , env.h_manhattan())
                counter_prior += 1
                heapq.heappush(q , (d[current_string][2] + d[current_string][3], counter_prior, np.copy(env.mat) ))
                frontier.append(current_string)
                
            env.up(1)
            
        
        
        
  
    if (ans == False):
        print("No Solution")
    else:
        
        current_string = np.array_str(env.mat)
        path = []
        path_matrix = []
        #print("Current string : " , current_string)
        
        while not d[current_string][0] == "-1":
            path.insert(0 , d[current_string][1])
            path_matrix.insert(0 , d[current_string][0])
            current_string = d[current_string][0]
        
        for x , y in zip (path_matrix , path):
            print(x , y)
        print(env.mat)
        


In [11]:
heuristics()

Number of steps :  1
Number of steps :  2
Number of steps :  3
Number of steps :  4
Number of steps :  5
Number of steps :  6
Number of steps :  7
Number of steps :  8
Number of steps :  9
Number of steps :  10
[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 12. 11.]
 [13. 14. 15. 16.]] up b0
[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 15. 11.]
 [13. 14. 12. 16.]] left b1
[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 15. 11.]
 [13. 14. 16. 12.]] right b0
[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 15.]
 [13. 14. 16. 12.]] down b0
[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]
 [13. 14. 16. 15.]]
